In [ ]:
!pip install 'scanpy == 1.9.1'
!pip install 'matplotlib == 3.6'


In [ ]:
import os
import sys
import scanpy as sc
import shutil
import matplotlib
import matplotlib.pyplot as plt


In [ ]:
#True/ Relative / Non Distinct Genes

def True_Relative_Non_Distinct_Unique_Genes_Constructor(all_timepoint_unique_genes_dict, timepoints_anndata, timepoints_anndata_names):
  print('Average Runtime: 16 minutes \n')
  all_timepoint_true_relative_distinct_unique_genes_dict = {}
  all_timepoint_all_unique_genes_dict = {}

  # Iterates through all four timepoints
  for timepoint_name, anndata in zip(timepoints_anndata_names, timepoints_anndata):

    unique_dicts = all_timepoint_unique_genes_dict.get(timepoint_name)
    all_clusters_curr_timepoint_True_Relative_Distinct_Unique_Genes = {}
    all_clusters_curr_timepoint_All_Unique_Genes = {}

    # Iterates through all clusters within each timepoint
    for clust in unique_dicts:

      # Obtains the unique gene tuples (gene, ratio metric gene ranking) from all timepoint unique genes, corresponding to the cluster and timepoint
      gene_unique_tuples = unique_dicts.get(clust)
      list_of_unique_candidate_distinct_genes = []
      true_distinct_genes = []
      relative_distinct_genes = []
      all_distinct_genes = []

      # for each gene within the unique gene tuples, the mean expression value is calculated
      for gene_tup in gene_unique_tuples:
        mean_exp = float((anndata.raw[anndata.obs['E_I_M'].isin([clust]),:][:,gene_tup[0]].X > 0).mean(0)[0][0])

        # Makes sure to filter out only genes expressed in greater than 10% of cells
        if mean_exp >= .1:
          list_of_unique_candidate_distinct_genes.append(gene_tup[0])

      # for all the filtered genes the percentage of said genes expression are calculated for the corresponding cluster
      for x, gene in enumerate(list_of_unique_candidate_distinct_genes):
        print('\rWorking on     '+'Timepoint:  '+ timepoint_name+'     Cluster:  '+clust+'     ' +'Gene:  '+ gene+'   '+ str(x + 1) + ':' + str(len(list_of_unique_candidate_distinct_genes)), end='')
        curr_clust_mean_exp = float((anndata.raw[anndata.obs['E_I_M'].isin([str(clust)]),:][:,gene].X > 0).mean(0)[0][0])
        all_other_cluster_mean_exp_list = []

        # for all clusters not said cluster the percentage is calculated
        for other_cluster in unique_dicts:
          if other_cluster != clust:

            other_clust_mean_exp = float((anndata.raw[anndata.obs['E_I_M'].isin([str(other_cluster)]),:][:,gene].X > 0).mean(0)[0][0])

            all_other_cluster_mean_exp_list.append(other_clust_mean_exp)

        # checks to see if current cluster gene percentage is greater than 35%
        if curr_clust_mean_exp >= .50:
          # checks to see if the second highest cluster gene percentage is at least half of less the percentage of the current cluster gene percentage
          second_highest_mean_exp = max(all_other_cluster_mean_exp_list)
          if curr_clust_mean_exp >= (second_highest_mean_exp *2):
            true_distinct_genes.append(gene)
        # checks to see if current cluster gene percentage is greater than 10% and less than 35%
        elif (curr_clust_mean_exp >= .10) and (curr_clust_mean_exp < .50):
          # checks to see if the second highest cluster gene percentage is at least half of less the percentage of the current cluster gene percentage
          second_highest_mean_exp = max(all_other_cluster_mean_exp_list)
          if curr_clust_mean_exp >= (second_highest_mean_exp *2):
            relative_distinct_genes.append(gene)
        # checks to see if current cluster gene percentage is greater than 10% meant to add to the dictionary that includes true and rel distinct genes
        if (curr_clust_mean_exp >= .10):
          # checks to see if the second highest cluster gene percentage is at least half of less the percentage of the current cluster gene percentage
          second_highest_mean_exp = max(all_other_cluster_mean_exp_list)
          if curr_clust_mean_exp >= (second_highest_mean_exp *2):
            all_distinct_genes.append(gene)

      # All true, relative, and non distinct genes are added to their corresponding clusters and a True, Relative, Non distinct title is added
      # For clusters with neither true or relative genes, no genes are added but a Non distinct title is added
      # Furthermore all true/relative combined genes are added to the all_clusters_curr_timepoint_All_Unique_Genes dictionary and the all_timepoint_all_unique_genes_dict
      if true_distinct_genes:
        all_clusters_curr_timepoint_True_Relative_Distinct_Unique_Genes[clust] = ('True Distinct', true_distinct_genes)

      elif relative_distinct_genes:
        all_clusters_curr_timepoint_True_Relative_Distinct_Unique_Genes[clust] = ('Relative Distinct', relative_distinct_genes)

      else:
        all_clusters_curr_timepoint_True_Relative_Distinct_Unique_Genes[clust] = ('Non Distinct', [])

      if all_distinct_genes:
        all_clusters_curr_timepoint_All_Unique_Genes[clust] = all_distinct_genes

    all_timepoint_true_relative_distinct_unique_genes_dict[timepoint_name] = all_clusters_curr_timepoint_True_Relative_Distinct_Unique_Genes
    all_timepoint_all_unique_genes_dict[timepoint_name] = all_clusters_curr_timepoint_All_Unique_Genes
    print('\n')
  return all_timepoint_true_relative_distinct_unique_genes_dict, all_timepoint_all_unique_genes_dict

In [ ]:
# Converts timepoint name with a / to an _ in order to allow directory transfer
def time_path(timepoint_name):

  time = ''

  if '/' in timepoint_name:
    time = timepoint_name.replace('/','_')

  else:
    time = timepoint_name

  return time

In [ ]:
def All_Unique_Genes_Dotplots_Constructor(cluster_distinction_dicts, anndata, current_timepoint_path, timepoint_name, time, all_clusters_ordered):

  for clust in cluster_distinction_dicts:

    distinction_gene_tuple = cluster_distinction_dicts.get(clust)
    distinction = distinction_gene_tuple[0]
    gene_list = distinction_gene_tuple[1]

    if distinction != 'Non Distinct':

      distinction_path = current_timepoint_path+'/'+distinction

      if not os.path.exists(distinction_path):
        os.mkdir(distinction_path)

      all_clust_all_genes = distinction_path+'/'+distinction+'_All_Genes'

      if not os.path.exists(all_clust_all_genes):
        os.mkdir(all_clust_all_genes)

      sc.settings.figdir = all_clust_all_genes

      sc.pl.dotplot(anndata, gene_list, title= distinction + '\n' + timepoint_name+'\n'+'Cluster: '+ clust,  mean_only_expressed=True, groupby='E_I_M',  categories_order = all_clusters_ordered, save=time+'_'+clust+'.png');

In [ ]:
def True_Relative_All_Top_Genes_Sorted_Constructor(cluster_distinction_dicts):

  true_distinct_top_genes_list = []
  relative_distinct_top_genes_list = []
  all_clusters_ordered = []

  for clust in cluster_distinction_dicts:

    distinction_gene_tuple_temp = cluster_distinction_dicts.get(clust)

    if distinction_gene_tuple_temp[0] == 'True Distinct':
      gene_list_temp = distinction_gene_tuple_temp[1]
      true_distinct_top_genes_list.append((clust, gene_list_temp))

    if distinction_gene_tuple_temp[0] == 'Relative Distinct':
      gene_list_temp = distinction_gene_tuple_temp[1]
      relative_distinct_top_genes_list.append((clust, gene_list_temp))
    all_clusters_ordered.append(clust)
  # was x: int(x[0]) before...
  true_distinct_top_genes_list_sorted = sorted(true_distinct_top_genes_list, key=lambda x: x[0])
  relative_distinct_top_genes_list_sorted = sorted(relative_distinct_top_genes_list, key=lambda x: x[0])

  all_clusters_ordered = sorted(all_clusters_ordered)

  all_combined_top_genes = true_distinct_top_genes_list

  for rel_tup in relative_distinct_top_genes_list:
    all_combined_top_genes.append(rel_tup)

  all_combined_top_genes_sorted =  sorted(all_combined_top_genes, key=lambda x: x[0])

  return true_distinct_top_genes_list_sorted, relative_distinct_top_genes_list_sorted, all_combined_top_genes_sorted, all_clusters_ordered

In [ ]:
def True_Relative_All_Top_Genes_Ordered_Constructor(all_combined_top_genes_sorted, true_distinct_top_genes_list_sorted, relative_distinct_top_genes_list_sorted):

  all_combined_ordered_genes = []
  true_distinct_ordered_genes = []
  relative_distinct_ordered_genes = []

  for top_gene_tup_all in all_combined_top_genes_sorted:
    all_genes = top_gene_tup_all[1][0:5]
    for top_gene_all in all_genes:
      all_combined_ordered_genes.append(top_gene_all)

  for top_gene_tup_true in true_distinct_top_genes_list_sorted:
    true_genes = top_gene_tup_true[1][0:5]
    for top_gene_true in true_genes:
      true_distinct_ordered_genes.append(top_gene_true)

  for top_gene_tup_rel in relative_distinct_top_genes_list_sorted:
    rel_genes = top_gene_tup_rel[1][0:5]
    for top_gene_rel in rel_genes:
      relative_distinct_ordered_genes.append(top_gene_rel)

  return true_distinct_ordered_genes, relative_distinct_ordered_genes, all_combined_ordered_genes

In [ ]:
def True_Relative_All_Top_Genes_Dotplots_Constructor(all_combined_ordered_genes, true_distinct_ordered_genes, relative_distinct_ordered_genes, current_timepoint_path, anndata, timepoint_name, time, all_clusters_ordered):

  true_distinction_path = current_timepoint_path+'/'+'True Distinct'

  relative_distinction_path = current_timepoint_path+'/'+'Relative Distinct'

  all_combined_path = current_timepoint_path

  sc.settings.figdir = all_combined_path

  sc.pl.dotplot(anndata, all_combined_ordered_genes, title= 'Top True and Relative Distinct Genes Combined' + '\n' + timepoint_name,  mean_only_expressed=True, groupby='E_I_M',  categories_order = all_clusters_ordered, save=time+' Top True_Relative Distinct Genes'+'.png');

  if not os.path.exists(true_distinction_path):
      os.mkdir(true_distinction_path)

  sc.settings.figdir = true_distinction_path

  if true_distinct_ordered_genes:
    sc.pl.dotplot(anndata, true_distinct_ordered_genes, title= 'Top True Distinct Genes' + '\n' + timepoint_name,  mean_only_expressed=True, groupby='E_I_M',  categories_order = all_clusters_ordered, save=time+' Top True Distinct Genes'+'.png');

  if not os.path.exists(true_distinction_path):
      os.mkdir(true_distinction_path)

  sc.settings.figdir = relative_distinction_path

  if relative_distinct_ordered_genes:
    sc.pl.dotplot(anndata, relative_distinct_ordered_genes, title= 'Top Relative Distinct Genes' + '\n' + timepoint_name,  mean_only_expressed=True, groupby='E_I_M',  categories_order = all_clusters_ordered, save=time+' Top Relative Distinct Genes'+'.png');

  return all_combined_path

In [ ]:
def True_Relative_Non_Distinct_Clusters_List_Constructor(cluster_distinction_dicts):

  list_of_clusters = cluster_distinction_dicts.keys()

  new_list_of_clusters_int = []

  for clus in list_of_clusters:
    # was int(clust) before to allow for numerical ordering
    new_list_of_clusters_int.append(clus)

  new_list_of_clusters_int_sorted = sorted(new_list_of_clusters_int)

  Non_distinct_clusters = []
  True_distinct_clusters = []
  Relative_distinct_clusters = []

  for clus in new_list_of_clusters_int_sorted:

    gene_tuple_distinction = cluster_distinction_dicts.get(str(clus))

    if gene_tuple_distinction[0] == 'Non Distinct':
      non_dist = str(clus)
      Non_distinct_clusters.append(non_dist)

    if gene_tuple_distinction[0] == 'True Distinct':
      true_dist = str(clus)
      True_distinct_clusters.append(true_dist)

    if gene_tuple_distinction[0] == 'Relative Distinct':
      rel_dist = str(clus)
      Relative_distinct_clusters.append(rel_dist)

  return True_distinct_clusters, Relative_distinct_clusters, Non_distinct_clusters

In [ ]:
def Cluster_Distinction_Text_File_Writer(all_combined_path, time, cluster_distinction_dicts, True_distinct_clusters, Relative_distinct_clusters, Non_distinct_clusters):

  distinct_dir = all_combined_path+'/'+'Cluster Distinctions.txt'
  distinct_file = open(distinct_dir,"w")
  distinct_file.write(time+':'+'\n\n')


  distinct_file.write('True Distinct Clusters: \n\n')

  for true in True_distinct_clusters:
    distinct_file.write('Cluster: '+str(true)+'\n')
  distinct_file.write('\n\n')

  distinct_file.write('Relative Distinct Clusters: \n\n')

  for rel in Relative_distinct_clusters:
    distinct_file.write('Cluster: '+str(rel)+'\n')
  distinct_file.write('\n\n')

  distinct_file.write('Non Distinct Clusters: \n\n')

  for non in Non_distinct_clusters:
    distinct_file.write('Cluster: '+str(non)+'\n')
  distinct_file.write('\n\n')

  distinct_file.close()

In [ ]:
def True_Relative_Non_Distinct_Data_Output(all_timepoint_true_relative_distinct_unique_genes_dict, timepoints_anndata, cluster_distinction_path, timepoints_anndata_names):

  all_timepoint_distinction_dict = {}

  for timepoint_name, anndata in zip(timepoints_anndata_names, timepoints_anndata):

    cluster_distinction_dicts = all_timepoint_true_relative_distinct_unique_genes_dict.get(timepoint_name)

    time = time_path(timepoint_name)

    current_timepoint_path = cluster_distinction_path + '/'+time

    if not os.path.exists(current_timepoint_path):
      os.mkdir(current_timepoint_path)


    true_distinct_top_genes_list_sorted, relative_distinct_top_genes_list_sorted, all_combined_top_genes_sorted, all_clusters_ordered = True_Relative_All_Top_Genes_Sorted_Constructor(cluster_distinction_dicts)

    All_Unique_Genes_Dotplots_Constructor(cluster_distinction_dicts, anndata, current_timepoint_path, timepoint_name, time, all_clusters_ordered)

    true_distinct_ordered_genes, relative_distinct_ordered_genes, all_combined_ordered_genes = True_Relative_All_Top_Genes_Ordered_Constructor(all_combined_top_genes_sorted, true_distinct_top_genes_list_sorted, relative_distinct_top_genes_list_sorted)

    all_combined_path = True_Relative_All_Top_Genes_Dotplots_Constructor(all_combined_ordered_genes, true_distinct_ordered_genes, relative_distinct_ordered_genes, current_timepoint_path, anndata, timepoint_name, time, all_clusters_ordered)

    True_distinct_clusters, Relative_distinct_clusters, Non_distinct_clusters = True_Relative_Non_Distinct_Clusters_List_Constructor(cluster_distinction_dicts)

    Cluster_Distinction_Text_File_Writer(all_combined_path, time, cluster_distinction_dicts, True_distinct_clusters, Relative_distinct_clusters, Non_distinct_clusters)

    all_timepoint_distinction_dict[timepoint_name] = (True_distinct_clusters, Relative_distinct_clusters, Non_distinct_clusters)

  return all_timepoint_distinction_dict